# CAPSTONE PROJECT THE BATTLE OF NEIGHBORHOODS (WEEK2)

AS I HAVE MENTIONED IN THE FIRST WEEK'S DATA THAT I WILL BE DEALING WITH FINDING THE BEST LOCATION FOR A PERSON WHO WANTS TO START HIS/HER OWN RETAIL BUSINESS ,WHERE FINDING THE ACQUIRED LOCATION FOR THE RETAIL SHOP PLAYS A MAJOR ROLE FOR HIS/HER BUSINESS TO EXPAND.

IN THIS PROJECT I WILL BE DEALING WITH 'NEWYORK DATA' FOR FINDING THE LOCATION TO OPEN A RETAIL SHOP.
MORE THE VENUE A LOCALITY HAS , THAT IN TURN WILL ATTRACT MORE CROWD AND WILL BE ONE OF THE BEST PLACE TO OPEN A RETAIL SHOP.

# IMPORTING REQUIRED LIBRARIES

In [1]:
!pip install geopy
!conda install -c conda-forge folium=0.5.0 --yes 

import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

%matplotlib inline 
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
mpl.style.use('ggplot')
from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
         

## NOW LET'S DOWNLOAD AND EXPLORE OUR DATASET

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

AS WE WOULD REQUIRE ONLY SOME COLUMNS 

In [6]:
neighborhoods_data = newyork_data['features']

In [4]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

NOW LET'S LOOK AT OUR DATA

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Geopy library to get the latitude and longitude values of New York City.

In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Let's assume that our customer wants to open his/her retail store in Bronx .
So, now we will examine that which neighborhood of Bronx will be the best location to open a retail shop.

In [10]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.85048545, -73.8404035580209.


In [12]:
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

## Defining Foursquare credentials

In [13]:
CLIENT_ID = 'RGPQKGEHBNDBLTLOPNAH1UIMBJU5NEAIGF233HQOKTCVGSAH' # your Foursquare ID
CLIENT_SECRET = 'KIUCG1P2ZA4NBXEIYMSXLPYCOCFQTKWPSWWDSWAXQ4UB5S4V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


LET'S EXPLORE ALL THE NEIGHBORHOODDS IN BRONX

In [14]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [16]:
print(bronx_venues.shape)
bronx_venues.head()

(1264, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


Let's check how many venues return for each neighborhood

In [31]:
df=bronx_venues.groupby('Neighborhood').count()
df

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,31,31,31,31,31,31
Baychester,22,22,22,22,22,22
Bedford Park,38,38,38,38,38,38
Belmont,96,96,96,96,96,96
Bronxdale,17,17,17,17,17,17
Castle Hill,9,9,9,9,9,9
City Island,23,23,23,23,23,23
Claremont Village,13,13,13,13,13,13
Clason Point,11,11,11,11,11,11


In [32]:
df.sort_values('Venue Category',ascending=False,inplace=True)

In [33]:
df

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belmont,96,96,96,96,96,96
Fordham,92,92,92,92,92,92
Kingsbridge,72,72,72,72,72,72
Pelham Bay,41,41,41,41,41,41
Bedford Park,38,38,38,38,38,38
Parkchester,38,38,38,38,38,38
Concourse Village,36,36,36,36,36,36
Kingsbridge Heights,35,35,35,35,35,35
Westchester Square,32,32,32,32,32,32


## From above Data Frame we can select the best neighborhood for our customer to open his/her Retail business from.
 ## As per our Data Frame 'BELMONT' seems to have the most numbers of venues, but 'FORDHAM' and 'KINGSBRIDGE' could aslo be suggested as a good neighborhood to start retail business from Bronx.

HENSE WE HAVE SOLVED THE PROBLEM DISCUSSED BY US IN OUR FIRST WEEK OF "THE BATTLE OF NEIGHBORHOOD" TO FIND THE BEST LOCATION FOR ONE OF THE BOROUGH,FROM WHERE A RETAIL SHOP/BUSINESS IF STARTED MIGHT BE BENEFICIAL.